In [ ]:
import pandas as pd
import sys
pd.set_option('display.max_rows', 100)

In [ ]:
sys.path.append('..\\packages')

In [ ]:
from suppression_check import DataAnonymizer

# CSV file with PreRedacted Data

In [ ]:
file_path = r'..\data\GraduationELLGender.csv'

# Load the CSV file into a DataFrame

In [ ]:
df_data = pd.read_csv(file_path)

# Instantiate the anonymizer

In [ ]:
organization_list = ['ParentEntity', 'ChildEntity']

In [ ]:
sensitive_list = ['Subgroup1', 'Subgroup2']

In [ ]:
frequency_value = 'GraduationCount'

In [ ]:
anonymizer = DataAnonymizer(df_data, organization_columns=organization_list, sensitive_columns=sensitive_list, frequency=frequency_value)

In [ ]:
df_log = anonymizer.create_log()

In [ ]:
display(df_log)

In [ ]:
df_redact_threshold = anonymizer.less_than_threshold_not_zero()

In [ ]:
df_redact_threshold.loc[:, 'Overlapping'] = 0

In [ ]:
for organization_column in organization_list:
    for sensitive_column in sensitive_list:
        redacted_filter = df_redact_threshold['RedactBinary'] == 1
        null_filter = (~((df_redact_threshold[sensitive_column].isna()) & (df_redact_threshold[organization_column].isna())))
        filter = (redacted_filter & null_filter)
        df_redacted = df_redact_threshold[filter][[organization_column, sensitive_column]].drop_duplicates()
        df_redacted['OverlappingAdd'] = 1
        df_redact_threshold = df_redact_threshold.merge(df_redacted, on=[organization_column, sensitive_column], how='left')
        df_redact_threshold['OverlappingAdd'].fillna(0, inplace=True)
        df_redact_threshold.loc[~redacted_filter, 'Overlapping'] = df_redact_threshold['Overlapping'] + df_redact_threshold['OverlappingAdd']
        df_redact_threshold.drop('OverlappingAdd', axis=1, inplace=True)
for sensitive_column in sensitive_list:
    redacted_filter = df_redact_threshold['RedactBinary'] == 1
    null_filter = df_redact_threshold[sensitive_column].notna()
    filter = (redacted_filter & null_filter)
    df_redacted = df_redact_threshold[filter][[sensitive_column]].drop_duplicates()
    df_redacted['OverlappingAdd'] = 1
    df_redact_threshold = df_redact_threshold.merge(df_redacted, on=[sensitive_column], how='left')
    df_redact_threshold['OverlappingAdd'].fillna(0, inplace=True)
    df_redact_threshold.loc[~redacted_filter, 'Overlapping'] = df_redact_threshold['Overlapping'] + df_redact_threshold['OverlappingAdd']
    df_redact_threshold.drop('OverlappingAdd', axis=1, inplace=True)
for organization_column in organization_list:
    redacted_filter = df_redact_threshold['RedactBinary'] == 1
    null_filter = df_redact_threshold[organization_column].notna()
    filter = (redacted_filter & null_filter)
    df_redacted = df_redact_threshold[filter][[organization_column]].drop_duplicates()
    df_redacted['OverlappingAdd'] = 1
    df_redact_threshold = df_redact_threshold.merge(df_redacted, on=[organization_column], how='left')
    df_redact_threshold['OverlappingAdd'].fillna(0, inplace=True)
    df_redact_threshold.loc[~redacted_filter, 'Overlapping'] = df_redact_threshold['Overlapping'] + df_redact_threshold['OverlappingAdd']
    df_redact_threshold.drop('OverlappingAdd', axis=1, inplace=True)
display(df_redact_threshold)

In [ ]:
df_min = df_redact_threshold[df_redact_threshold['RedactBinary'] != 1].groupby('RedactBinary').min().reset_index()
minimum_value = df_min['Overlapping'].loc[0]
df_max = df_redact_threshold[df_redact_threshold['RedactBinary'] != 1].groupby('RedactBinary').max().reset_index()
display(df_max)
max_value = df_max['Overlapping'].loc[0]
print(minimum_value)
print(max_value)

In [ ]:
print(minimum_value)
print(max_value)

In [ ]:
mask = (self.df_log['Overlapping'] == self.df_log['Overlapping'].max()) & (self.df_log['RedactBinary'] == 0)

self.df_log.loc[mask, 'Redact'] = 'Overlapping threshold secondary suppression'

self.df_log.loc[mask, 'RedactBinary'] = 1

In [ ]:
df_one_count = anonymizer.one_count_redacted()

In [ ]:
display(df_one_count)